In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
import os

In [2]:
#reading the document dataframe which cluster it belongs to
#import the document_clustered.csv file
dataFrame = pd.read_csv('document_clustered.csv',sep=',',index_col=None)

In [3]:
dataFrame.head()

,category,content,clusters,document
0,0,alfonsin allsuite arnott bonded briefed bulax ...,2,0000045
1,0,dependent cellulosa allentown bases ergenc err...,23,0008858
2,0,cellulosa emphasizing edina celsius dependent ...,23,0001242
3,0,edina cellulosa allentown emphasizing celsius ...,23,0009002
4,0,downed ecologists edina emphasizing cellulosa ...,23,0009026


In [4]:
#lets train the model with KNeighbourclassifier
X= dataFrame['content']
y= dataFrame['clusters'] #type is pandas.core.series.Series

In [5]:
#import necessary libraries from scikit learn
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
import re

In [6]:
def tokenize(text):
    words = [words.lower() for words in nltk.word_tokenize(text)]
    filter_tokens = []
    for tokens in words:
        if re.search('[a-zA-z]',tokens):
            filter_tokens.append(tokens)
    return filter_tokens

In [7]:
#TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=1.0,max_features=15000,
                                   use_idf=True,tokenizer=tokenize,ngram_range=(1,3))
%time tfidf_matrix = tfidf_vectorizer.fit_transform(X)
print(tfidf_matrix.shape)

CPU times: user 3.49 s, sys: 12 ms, total: 3.5 s
Wall time: 3.68 s
(11475, 14240)


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.3)

In [9]:
print(len(X_train),len(X_test),len(Y_train),len(Y_test)) #total dataset is splitted as 80,70

8032 3443 8032 3443


In [10]:
#Pipeline
knn_clf = Pipeline([('vect',TfidfVectorizer()),('clf',KNeighborsClassifier(n_neighbors=50))])
%time knn_fit = knn_clf.fit(X_train,Y_train).predict(X_test)
accuracy = np.mean(knn_fit == Y_test)

CPU times: user 1.04 s, sys: 48 ms, total: 1.09 s
Wall time: 1.09 s


In [11]:
print("Accuracy using KNeighbors is {}".format(accuracy * 100))

Accuracy using KNeighbors is 66.047051989544


In [12]:
#Random Forest Classifier
from sklearn .ensemble import RandomForestClassifier
%time rndm_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',RandomForestClassifier(n_estimators=20,criterion='gini',oob_score=True))])

rndm_predict = rndm_clf.fit(X_train,Y_train).predict(X_test)
rndm_score = np.mean(rndm_predict==Y_test)

print("Accuracy using Random Forest Classifier is:{}".format(rndm_score * 100))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 260 µs
Accuracy using Random Forest Classifier is:100.0


/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


In [14]:
#save the model
from sklearn.externals import joblib
joblib.dump(rndm_clf,'cluster_classification_model.pkl')

['cluster_classification_model.pkl']

In [19]:
#load the saved model
#
trained_model = joblib.load('cluster_classification_model.pkl')

array([48])